<h1><b><center>How to use this file with PyRat?</center></b></h1>

Google Colab provides an efficient environment for writing codes collaboratively with your group. For us, teachers, it allows to come and see your advancement from time to time, and help you solve some bugs if needed.

The PyRat software is a complex environment that takes as an input an AI file (as this file). It requires some resources as well as a few Python libraries, so we have installed it on a virtual machine for you.

PyRat is a local program, and Google Colab is a distant tool. Therefore, we need to indicate the PyRat software where to get your code! In order to submit your program to PyRat, you should follow the following steps:

1.   In this notebook, click on *Share* (top right corner of the navigator). Then, change sharing method to *Anyone with the link*, and copy the sharing link;
2.   On the machine where the PyRat software is installed, start a terminal and navigate to your PyRat directory;
3.   Run the command `python ./pyrat.py --rat "<link>" <options>`, where `<link>` is the share link copied in step 1. (put it between quotes), and `<options>` are other PyRat options you may need.

<h1><b><center>Pre-defined constants</center></b></h1>

A PyRat program should -- at each turn -- decide in which direction to move. This is done in the `turn` function later in this document, which should return one of the following constants:

In [ ]:
MOVE_DOWN = 'D'
MOVE_LEFT = 'L'
MOVE_RIGHT = 'R'
MOVE_UP = 'U'

<h1><b><center>Your coding area</center></b></h1>

Please put your functions, imports, constants, etc. between this text and the PyRat functions below. You can add as many code cells as you want, we just ask that you keep things organized (one function per cell, commented, etc.), so that it is easier for the teachers to help you debug your code!

In [ ]:
#Importation module
import random as rd
import heapq
import multiprocessing

#python pyrat.py --rat "" --python "https://colab.research.google.com/drive/1tSogjcQWO2cdv_WcRPsMTqhfp5cWVnFE?usp=sharing" --tests 100 --nodrawing -x 10 -y 10 -p 6 -md 0.0 --synchronous
#python pyrat.py --rat "https://colab.research.google.com/drive/1tSogjcQWO2cdv_WcRPsMTqhfp5cWVnFE?usp=sharing" -md 0.0 -p 1 -x 5 -y 5 --random_seed 1
#python pyrat.py --rat "https://colab.research.google.com/drive/1FLItxuxxs9-53U7gVDPxSxR3OlYIh1FS?usp=sharing" -p 5 -x 15 -y 15 -d 0.5 --random_seed 1
#python pyrat.py --rat "https://colab.research.google.com/drive/1FLItxuxxs9-53U7gVDPxSxR3OlYIh1FS?usp=sharing" -p 4 -x 7 -y 7 -d 0.0 --random_seed 1

In [ ]:
#Variable global
meta_graph = dict()
routes = list()
best_path = list()
best_length = 1e40
index_route=-1

In [ ]:
def move_from_locations(source_location, target_location):

  difference = (target_location[0] - source_location[0], target_location[1] - source_location[1])
  if difference == (0, -1):
    return MOVE_DOWN
  elif difference == (0, 1):
    return MOVE_UP
  elif difference == (1, 0):
    return MOVE_RIGHT
  elif difference == (-1, 0):
    return MOVE_LEFT
  else :
    raise Exception("Impossible move")

In [ ]:
def dijkstra(player_location, maze_map):
  #initialize
  priority_queue = list()
  visited_locations=list()
  heapq.heappush(priority_queue, (0, (player_location, None)))
  routing_table = dict()
  #algorithm loop
  while priority_queue != list() and len(routing_table)!=len(maze_map):
    distance, vertices = heapq.heappop(priority_queue)
    visited_locations.append(vertices[0])
    routing_table[vertices[0]] = vertices[1]
    #Parcours les voisins de v[0] = current_location
    for neighbor in maze_map[vertices[0]].keys():
      distance_of_neighbor = distance + maze_map[vertices[0]][neighbor]
      if neighbor not in visited_locations:
        heapq.heappush(priority_queue, (distance_of_neighbor, (neighbor, vertices[0])))

  return routing_table

In [ ]:
def finds_route(maze_map, routing_table, source_location, target_location):
  '''Renvoie la distance et la route de source_location à targe_location dans une liste de positions'''
  route = list()
  location = target_location
  distance  = 0
  #On parcours le routing_table à l'envers à partir du fromage jusqu'au source_location
  while location != source_location:
    route.insert(0, location)
    ##print(source_location,location,routing_table[location])
    distance += maze_map[routing_table[location]][location]
    #Location devient son parent
    location = routing_table[location]

  route.insert(0, source_location)

  return route, distance

In [ ]:
def build_meta_graph(maze_map, center_location, pieces_of_cheese):
  '''Return the meta-graph and all necessary routing tables'''
  #Meta Graph est un dictionnaire de dictionnaire de tuple (distance, route_associé)
  #On cherche à créer un graph complèt ne contenant que des fromages et la position initiale

  global meta_graph
  locations = c.deepcopy(pieces_of_cheese)
  locations.insert(0, center_location)

  for k in range(len(locations)):
    #Calcul le routing_table à partir du vertices location
    routing_table=dijkstra(locations[k],maze_map)
    meta_graph[locations[k]]=dict()

    for i in range(len(locations)):
      if i != k:
        #Route_ki est la route réel à partir entre location[k] et location[i] et distance est sa distance totale
        route_ki, distance = finds_route(maze_map, routing_table, locations[k], locations[i])

        meta_graph[locations[k]][locations[i]] = (distance, route_ki)
      else:
        meta_graph[locations[k]][locations[i]] = (0,[])


In [ ]:
def meta_graph_route_to_route() :
  # Return the sequence of locations in the maze to perform a route in the meta-graph
  global routes
  global best_path
  global meta_graph

  for i in range(len(best_path)-1):
    routes += meta_graph[best_path[i]][best_path[i+1]][1][1:]


In [ ]:
def brute(remaining, current_position, path_so_far, weight_so_far):
  '''Actualise les variables globales best_lenght et best_path'''
  global best_length
  global best_path
  global meta_graph

  if remaining==[]:
      if weight_so_far < best_length:
          best_length = weight_so_far
          ##print(best_lenght)
          best_path = path_so_far
  else:
      for i in range(len(remaining)):
        brute(remaining[:i]+remaining[i+1:], remaining[i], path_so_far + [remaining[i]], weight_so_far + meta_graph[current_position][remaining[i]][0])

In [ ]:
def move(player_location,index):
  global routes
  return move_from_locations(routes[index], routes[index + 1])


<h1><b><center>PyRat functions</center></b></h1>

The `preprocessing` function is called at the very start of a game. It is attributed a longer time compared to `turn`, which allows you to perform intensive computations. If you store the results of these computations into **global** variables, you will be able to reuse them in the `turn` function.

*Input:*
*   `maze_map` -- **dict(pair(int, int), dict(pair(int, int), int))** -- The map of the maze where the game takes place. This structure associates each cell with the dictionry of its neighbors. In that dictionary of neighbors, keys are cell coordinates, and associated values the number of moves required to reach that neighbor. As an example, `list(maze_map[(0, 0)].keys())` returns the list of accessible cells from `(0, 0)`. Then, if for example `(0, 1)` belongs to that list, one can access the number of moves to go from `(0, 0)` to `(0, 1)` by the code `maze_map[(0, 0)][0, 1)]`.
*   `maze_width` -- **int** -- The width of the maze, in number of cells.
*   `maze_height` -- **int** -- The height of the maze, in number of cells.
*   `player_location` -- **pair(int, int)** -- The initial location of your character in the maze.
*   `opponent_location` -- **pair(int,int)** -- The initial location of your opponent's character in the maze.
*   `pieces_of_cheese` -- **list(pair(int, int))** -- The initial location of all pieces of cheese in the maze.
*   `time_allowed` -- **float** -- The time you can take for preprocessing before the game starts checking for moves.

*Output:*
*   This function does not output anything.

In [ ]:
def preprocessing (maze_map, maze_width, maze_height, player_location, opponent_location, pieces_of_cheese, time_allowed) :

    global meta_graph
    global routing_table
    global route
    # Example prints that appear in the shell only at the beginning of the game
    # Remove them when you write your own program

    build_meta_graph(maze_map, player_location, pieces_of_cheese)
    ##print('meta=',  meta_graph)
    remaining = list(meta_graph.keys())
    brute(remaining, player_location, list(), 0)
    meta_graph_route_to_route()
    routes.insert(0,player_location)
    ##print('routes=', routes)

The `turn` function is called each time the game is waiting
for the player to make a decision (*i.e.*, to return a move among those defined above).

*Input:*
*   `maze_map` -- **dict(pair(int, int), dict(pair(int, int), int))** -- The map of the maze. It is the same as in the `preprocessing` function, just given here again for convenience.
*   `maze_width` -- **int** -- The width of the maze, in number of cells.
*   `maze_height` -- **int** -- The height of the maze, in number of cells.
*   `player_location` -- **pair(int, int)** -- The current location of your character in the maze.
*   `opponent_location` -- **pair(int,int)** -- The current location of your opponent's character in the maze.
*   `player_score` -- **float** -- Your current score.
*   `opponent_score` -- **float** -- The opponent's current score.
*   `pieces_of_cheese` -- **list(pair(int, int))** -- The location of remaining pieces of cheese in the maze.
*   `time_allowed` -- **float** -- The time you can take to return a move to apply before another time starts automatically.

*Output:*
*   A chosen move among `MOVE_UP`, `MOVE_DOWN`, `MOVE_LEFT` or `MOVE_RIGHT`.

In [ ]:
def turn (maze_map, maze_width, maze_height, player_location, opponent_location, player_score, opponent_score, pieces_of_cheese, time_allowed) :
  global index_route
  index_route+=1
  return move(player_location,index_route)